In [ ]:
import pandas as pd
import matplotlib.pylab as plt
import surprise
import numpy as np

In [ ]:
# Read csv
trade_df = pd.read_csv("./data/trade.csv", encoding="utf-8", parse_dates=['ISSUEDT'],
                       dtype={'HSCD': str, "QTY": float, 'BYRADDR2': str, 'SPLYADDR2': str})
trade_df.head()

In [ ]:
trade_df.columns

In [ ]:
trade_df[['BYRORGNM1', 'SPLYORGNM1', 'AMT', 'CUR']].head()

In [ ]:
trade_df = trade_df[['BYRORGNM1', 'SPLYORGNM1', 'AMT', 'CUR']]

In [ ]:
trade_df.CUR.drop_duplicates()

In [ ]:
trade_df = trade_df.loc[(trade_df.CUR == "USD") | (trade_df.CUR == "KRW")]

In [ ]:
trade_df.loc[trade_df.CUR == 'USD'].head()

In [ ]:
trade_df.loc[trade_df.CUR == 'KRW'].head()

In [ ]:
def calculate_krw(amt: float, cur: str) -> float:
    if cur == 'KRW':
        return amt
    elif cur == 'USD':
        return amt * 1244

In [ ]:
trade_df['AMT_KRW'] = trade_df.apply(lambda x: calculate_krw(x['AMT'], x['CUR']), axis=1)

In [ ]:
trade_df.loc[[0, 35800]]

In [ ]:
df = trade_df[['SPLYORGNM1', 'BYRORGNM1', 'AMT_KRW']]

In [ ]:
df.head()

In [ ]:
df = df.groupby(['SPLYORGNM1', 'BYRORGNM1']).agg('sum').sort_values('AMT_KRW', ascending=False).reset_index()

In [ ]:
df.head()

In [ ]:
df.columns = ['spl', 'byr', 'krw']

In [ ]:
df.shape

In [ ]:
df_table = df.set_index(["spl", "byr"]).unstack()
df_table.shape

In [ ]:
plt.imshow(df_table)
plt.grid(False)
plt.xlabel("byr")
plt.ylabel("spl")
plt.title("Trade Matrix")
plt.show()

In [ ]:
list(df_table.index).index('POSCO')

In [ ]:
val = '주식회사 포스코대우'
[(index, row.index(val)) for index, row in enumerate(list(df_table.columns)) if val in row]

In [ ]:
df_table.iloc[13660:13670, 13740:13750]

In [ ]:
df_table.count(axis=1).sort_values(ascending=False)[:100].plot.box

In [ ]:
plt.imshow(df_table.loc[df_table.count(axis=1).sort_values(ascending=False)[:100].index])
plt.grid(False)
plt.xlabel("byr")
plt.ylabel("spl")
plt.title("Trade Matrix")
plt.show()

In [ ]:
plt.imshow(df_table.iloc[13660:13670, 13740:13750])
plt.grid(False)
plt.xlabel("byr")
plt.ylabel("spl")
plt.title("Trade Matrix")
plt.show()

In [ ]:
df_table.count(axis=0).sort_values(ascending=False)

## Make dataframe to surprise dataset

In [ ]:
df.krw.max()

In [ ]:
df.krw.min()

In [ ]:
from surprise import Reader
from surprise import Dataset

In [ ]:
reader = Reader(rating_scale)

In [ ]:
df.head()

In [ ]:
df.krw.max()

In [ ]:
reader=Reader(rating_scale=(0, 5))

In [ ]:
df

In [ ]:
df.krw = df.krw / df.krw.max() * 5.0

In [ ]:
data = Dataset.load_from_df(df[['spl', 'byr', 'krw']], reader=reader)

In [ ]:
data.raw_ratings

In [ ]:
from surprise.model_selection import KFold

bsl_options = {
    'method': 'als',
    'n_epochs': 500,
    'reg_u': 12,
    'reg_i': 5
}
algo = surprise.BaselineOnly(bsl_options)

np.random.seed(0)
acc = np.zeros(3)
cv = KFold(3)
for i, (trainset, testset) in enumerate(cv.split(data)):
    algo.fit(trainset)
    predictions = algo.test(testset)
    acc[i] = surprise.accuracy.rmse(predictions, verbose=True)
acc.mean()

In [ ]:
from surprise.model_selection import cross_validate

sim_options = {'name': 'msd'}
algo = surprise.KNNBasic(sim_options=sim_options)
cross_validate(algo, data)["test_mae"].mean()

In [ ]:
data_ml = surprise.Dataset.load_builtin('ml-100k')

In [ ]:
data.raw_ratings

In [ ]:
data_ml.raw_ratings